
# Risco de Crédito

## Financiamento de Veículos

### Primeira análise

- Criar um modelo que determina qual o risco de uma pessoa entrar em inadimplência caso ela contrate crédito comigo - PD (probability of default)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('/Users/rguedes/Desktop/home-credit-default-risk/application_train.csv')

In [ ]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


In [ ]:
df.shape

(246008, 122)

In [ ]:
df.tail()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
246003,242114,0,Cash loans,F,N,Y,1,270000.0,1172470.5,34411.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,8.0
246004,452374,0,Cash loans,F,N,Y,0,180000.0,654498.0,27859.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
246005,276545,1,Revolving loans,M,N,N,1,112500.0,270000.0,13500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
246006,236776,1,Cash loans,M,Y,N,3,202500.0,204858.0,17653.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
246007,454197,0,Cash loans,F,N,Y,2,81000.0,547344.0,23139.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
df.TARGET.value_counts(normalize = True, dropna = False)

0    0.918824
1    0.081176
Name: TARGET, dtype: float64

In [ ]:
var_num = df.dtypes['CODE_GENDER':][df.dtypes['CODE_GENDER':] != 'object'].index
var_num

Index(['CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=104)

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 1)
df_treino, df_teste = df_treino.copy(), df_teste.copy()

In [ ]:
df_treino.TARGET.value_counts(True)

0    0.919235
1    0.080765
Name: TARGET, dtype: float64

In [ ]:
df_teste.TARGET.value_counts(True)

0    0.917178
1    0.082822
Name: TARGET, dtype: float64

In [ ]:
x_treino = df_treino[var_num]
x_teste = df_teste[var_num]

y_treino = df_treino['TARGET']
y_teste = df_teste['TARGET']

In [ ]:
si = SimpleImputer(strategy = 'median')
si.fit(x_treino)
x_treino = pd.DataFrame(si.transform(x_treino), columns = x_treino.columns)
x_teste = pd.DataFrame(si.transform(x_teste), columns = x_teste.columns)

In [ ]:
%%time
modelo = DecisionTreeClassifier(max_depth = 8, random_state = 1)
modelo.fit(x_treino, y_treino)

CPU times: user 5.65 s, sys: 147 ms, total: 5.8 s
Wall time: 9.53 s


DecisionTreeClassifier(max_depth=8, random_state=1)

In [ ]:
y_pred_teste = modelo.predict_proba(x_teste)
y_pred_teste = y_pred_teste[:, 1]

In [ ]:
modelo.predict_proba(x_treino)

array([[0.95753947, 0.04246053],
       [0.94359783, 0.05640217],
       [0.93923323, 0.06076677],
       ...,
       [0.72526502, 0.27473498],
       [0.91443693, 0.08556307],
       [0.97174254, 0.02825746]])

In [ ]:
roc_auc_score(y_teste, y_pred_teste)

0.7086774815887704

## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
var_num = list(df.dtypes['CODE_GENDER':][df.dtypes['CODE_GENDER':] != 'object'].index)
var_edu = ['NAME_EDUCATION_TYPE']

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 1)
df_treino, df_teste = df_treino.copy(), df_teste.copy()

x_treino = df_treino[var_num + var_edu]
x_teste = df_teste[var_num + var_edu]

y_treino = df_treino['TARGET']
y_teste = df_teste['TARGET']

In [ ]:
pipe = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler()),
    ('modelo', KNeighborsClassifier())
])

In [ ]:
pipe.fit(x_treino, y_treino)

In [ ]:
y_pred_teste = pipe.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_pred_teste)

In [ ]:
y_pred_teste = pipe.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_pred_teste)

0.708786754654655

## Tratando Múltiplas Variáveis de Formas Distintas

In [ ]:
var_num = list(df.dtypes['CODE_GENDER':][df.dtypes['CODE_GENDER':] != 'object'].index)
var_edu = ['NAME_EDUCATION_TYPE']
var_cat = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'OCCUPATION_TYPE']

In [ ]:
df_treino, df_teste = train_test_split(df, test_size = 0.2, random_state = 1)
df_treino, df_teste = df_treino.copy(), df_teste.copy()

x_treino = df_treino[var_num + var_edu + var_cat]
x_teste = df_teste[var_num + var_edu + var_cat]

y_treino = df_treino['TARGET']
y_teste = df_teste['TARGET']

In [ ]:
pd.crosstab(df.TARGET, df.NAME_EDUCATION_TYPE, normalize = 'columns')

NAME_EDUCATION_TYPE,Academic degree,Higher education,Incomplete higher,Lower secondary,Secondary / secondary special
TARGET,,,,,
0,0.977444,0.9463,0.912768,0.887816,0.910193
1,0.022556,0.0537,0.087232,0.112184,0.089807


In [ ]:
def trata_educacao_(x):
    if x == 'Lower secondary':
        return 1
    elif x == 'Secondary / secondary special':
        return 2
    elif x == 'Incomplete higher':
        return 3
    elif x == 'Higher education':
        return 4
    elif x == 'Academic degree':
        return 5
    else:
        return -1
    
    
def trata_educacao(tabela):
    
    tabela = tabela.copy()
    
    for coluna in tabela.columns:
        if coluna == 'NAME_EDUCATION_TYPE':
            tabela[coluna] = tabela[coluna].apply(trata_educacao_)
            
    return tabela

In [ ]:
class TrataEducacao(TransformerMixin):
    
    def fit(self, X, y):
        ## fazer os devidos calculos em cima do X
        pass
        
    def transform(self, X):
        ## realiza a transformacao em cima do X
        pass

In [ ]:
x_treino

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_EDUCATION_TYPE
190148,0,94500.0,364896.0,16200.0,315000.0,0.020246,-22339,365243,-8079.0,-4831,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,Secondary / secondary special
189722,0,292500.0,478498.5,46741.5,454500.0,0.010006,-11802,-3676,-950.0,-4149,...,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,Secondary / secondary special
235486,0,81000.0,396171.0,21492.0,342000.0,0.031329,-8312,-425,-8288.0,-991,...,0,0,0,0.0,0.0,0.0,0.0,2.0,0.0,Secondary / secondary special
207577,0,117000.0,180000.0,18900.0,180000.0,0.031329,-11917,-1305,-5559.0,-4371,...,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,Secondary / secondary special
41697,0,238500.0,640080.0,29970.0,450000.0,0.046220,-8483,-1141,-6584.0,-1031,...,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0,Incomplete higher
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229119,1,103500.0,355536.0,14040.0,270000.0,0.030755,-13665,-381,-1688.0,-5510,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,Secondary / secondary special
5192,0,157500.0,630000.0,20452.5,630000.0,0.025164,-16955,-4012,-213.0,-483,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,Secondary / secondary special
208780,1,90000.0,523597.5,23193.0,468000.0,0.006852,-12842,-2297,-3436.0,-1986,...,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,Higher education
229611,0,180000.0,346500.0,18796.5,346500.0,0.008625,-21240,365243,-1555.0,-1555,...,0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,Higher education


In [ ]:
ohe = OneHotEncoder(drop = 'if_binary', sparse = False)
ohe.fit(x_treino[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
ohe.transform(x_treino[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])

array([[0., 1.],
       [1., 0.],
       [1., 1.],
       ...,
       [0., 0.],
       [0., 1.],
       [1., 1.]])

In [ ]:
pipe_num = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scaler', StandardScaler())
])

pipe_edu = Pipeline(steps = [
    ('educacao', FunctionTransformer(trata_educacao))
])

pipe_cat = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('ohe', OneHotEncoder(drop = 'if_binary', sparse = False))
    
])

tratar_variaveis = ColumnTransformer(transformers = [
    ('num', pipe_num, var_num),
    ('edu', pipe_edu, var_edu),
    ('cat', pipe_cat, var_cat)  
])

pipe = Pipeline(steps = [
    ('tratamento', tratar_variaveis),
    ('modelo', DecisionTreeClassifier(max_depth = 9, random_state = 1))
])

In [ ]:
pipe.fit(x_treino, y_treino)

Pipeline(steps=[('tratamento',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['CNT_CHILDREN',
                                                   'AMT_INCOME_TOTAL',
                                                   'AMT_CREDIT', 'AMT_ANNUITY',
                                                   'AMT_GOODS_PRICE',
                                                   'REGION_POPULATION_RELATIVE',
                                                   'DAYS_BIRTH',
                                                   'DAYS_EMPLOYED',
                                                   'DAYS_REGISTRA

In [ ]:
y_pred_teste = pipe.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_pred_teste)

0.7067811837376206

In [ ]:
y_pred_teste = pipe.predict_proba(x_teste)[:, 1]
roc_auc_score(y_teste, y_pred_teste)

0.7107944925983262

In [ ]:
x_treino.shape

(196806, 108)